In [3]:
# pip install yfinance
# !pip install snowflake-connector-python
# !pip install snowflake-snowpark-python

In [14]:
import yfinance as yf
from tqdm import tqdm
import pandas as pd

# from snowflake.connector.pandas_tools import write_pandas
# import pandas as pd
# import configparser
# from snowflake.snowpark.session import Session

# config = configparser.ConfigParser()
# config.read("snowflake_config_logs.ini")

# connection_params = {
#     'user': f'{config["Snowflake"]["user"]}',
#     'password': f'{config["Snowflake"]["password"]}',
#     'account': f'{config["Snowflake"]["account"]}',
#     'warehouse': f'{config["Snowflake"]["warehouse"]}',
#     'database': f'{config["Snowflake"]["database"]}',
#     'schema': f'{config["Snowflake"]["schema"]}',
#     'role': f'{config["Snowflake"]["role"]}'
# }
# #print('connection_successful')
# session = Session.builder.configs(connection_params).create()

stocks = {
  "Technology": [
    {"name": "Microsoft Corporation", "ticker": "MSFT"},
    {"name": "Apple Inc.", "ticker": "AAPL"},
    {"name": "Alphabet Inc.", "ticker": "GOOGL"},
    {"name": "Intel Corporation", "ticker": "INTC"},
    {"name": "Cisco Systems, Inc.", "ticker": "CSCO"}
  ],
  "Consumer Discretionary": [
    {"name": "Amazon.com, Inc.", "ticker": "AMZN"},
    {"name": "Tesla, Inc.", "ticker": "TSLA"},
    {"name": "The Home Depot, Inc.", "ticker": "HD"},
    {"name": "McDonald's Corporation", "ticker": "MCD"},
    {"name": "Nike, Inc.", "ticker": "NKE"}
  ],
  "Healthcare": [
    {"name": "Johnson & Johnson", "ticker": "JNJ"},
    {"name": "Pfizer Inc.", "ticker": "PFE"},
    {"name": "Merck & Co., Inc.", "ticker": "MRK"},
    {"name": "Abbott Laboratories", "ticker": "ABT"},
    {"name": "Amgen Inc.", "ticker": "AMGN"}
  ],
  "Financials": [
    {"name": "JPMorgan Chase & Co.", "ticker": "JPM"},
    {"name": "Bank of America Corporation", "ticker": "BAC"},
    {"name": "Wells Fargo & Company", "ticker": "WFC"},
    {"name": "Goldman Sachs Group, Inc.", "ticker": "GS"},
    {"name": "Morgan Stanley", "ticker": "MS"}
  ],
  "Industrials": [
    {"name": "Boeing Company", "ticker": "BA"},
    {"name": "Caterpillar Inc.", "ticker": "CAT"},
    {"name": "General Electric Company", "ticker": "GE"},
    {"name": "3M Company", "ticker": "MMM"},
    {"name": "Honeywell International Inc.", "ticker": "HON"}
  ],
  "Consumer Staples": [
    {"name": "The Procter & Gamble Company", "ticker": "PG"},
    {"name": "Coca-Cola Company", "ticker": "KO"},
    {"name": "PepsiCo, Inc.", "ticker": "PEP"},
    {"name": "Walmart Inc.", "ticker": "WMT"},
    {"name": "Costco Wholesale Corporation", "ticker": "COST"}
  ]
}


In [15]:
stocks_list = [st["ticker"]  for key,val in stocks.items() for st in val]
counter = 1
for ticker in tqdm(stocks_list):
    if counter == 1:
        data = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        data["stock"] = [ticker]*len(data)
    else:
        print(ticker)
        df = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        df["stock"] = [ticker]*len(df)
        data = data._append(df, ignore_index=True)
        del df
    counter += 1
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
 23%|██▎       | 7/30 [00:00<00:00, 31.28it/s]

AAPL
GOOGL
INTC
CSCO
AMZN
TSLA
HD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MCD
NKE
JNJ
PFE
MRK
ABT


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AMGN
JPM
BAC
WFC
GS
MS
BA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


CAT
GE
MMM
HON
PG


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 30/30 [00:01<00:00, 26.54it/s]

KO
PEP
WMT
COST


In [16]:
data = data[["stock","Close","Adj Close"]]

In [17]:
data.to_csv("stocks_data.csv", index=False)

In [21]:
required_info = ['city','state','zip','country',"industry", "sector", "fullTimeEmployees", "auditRisk", 'boardRisk','compensationRisk',
 'shareHolderRightsRisk','overallRisk', 'dividendRate','dividendYield','beta', 'totalCash',
 'totalCashPerShare', 'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare',
 'returnOnAssets', 'returnOnEquity',
 'freeCashflow', 'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins']

stock_info = {}
for ticker in tqdm(stocks_list):
    info = yf.Ticker(ticker)
    stock_info[ticker] = {k:v for k, v in info.info.items() if k in required_info}


100%|██████████| 30/30 [00:02<00:00, 14.46it/s]


In [25]:
import json

with open("stocks_info.json", "w") as fp:
    json.dump(stock_info, fp)